In [3]:
from vsrc.constants import DATA
from ultralytics import YOLO
from os import path, listdir, makedirs
from shutil import rmtree, copy
import random
import time
import cv2

### Load YOLOv8n Model

In [4]:
# Constants
IMAGE_SIZE = 128
PROJECT_NAME = "logs"
RECIPE_NAME = "resistor_values"
TRAIN_RUN = f"{RECIPE_NAME}_run-{time.strftime('%Y%m%d-%H%M%S')}"
DATA_PATH = "./datasets/partial/resistor"

# Model
MODEL_PATH = f"./models/pretrained/yolov8n-coco.pt"
TRAIN_PARAM = {
    # Model definition
    'data': f"./models/recipes/resistor_values.yaml",
    'resume': False,
    'device': '0',
    'pretrained': True,
    # Names
    'project' : PROJECT_NAME,
    'name': TRAIN_RUN,
    # Training Parameters
    'batch': 16,
    'imgsz': IMAGE_SIZE,
    'epochs': 20,
    'patience': 10,
    'cos_lr': True,
    # Augmentation
    'hsv_h': 0.05, # Higher than default for resistor
    'hsv_s': 0.3, # Colours should not change too much
    'hsv_v': 0.2, # Colours should not change too much
    'degrees': 5, # Rotation
    'translate': 0.0, # Translation
    'scale': 1.0, # Scaling - camera is always at the same distance
    'shear': 0.0, # Shearing
    'perspective': 0.0, # Perspective
    'flipud': 0.5, # Flip up-down
    'fliplr': 0.5, # Flip left-right
    'mosaic': 0.5, # Mosaic
    'mixup': 0.0, # Mixup
    'copy_paste': 0.0, # Copy-paste
    'crop_fraction': 1.0, # Crop fraction
    # Loss weights
    'cls' : 8.0, # Class 
    'box' : 2.0, # Box accuracy
    'dfl' : 1.5, # Help manage unbalanced classes
    # Post parameters
    'save': True,
    'save_period': 5,
    'plots': False,
    # Misc
    'verbose': False,
}

PATHS = {
    'train': f"{DATA_PATH}/training/images/train",
    'val': f"{DATA_PATH}/training/images/val",
    'test': f"{DATA_PATH}/training/images/test",
}

# Reorganise the data
PARAM = {
    'path' : DATA_PATH,
    'train' : 0.7,
    'val' : 0.2,
    'test' : 0.1,
}

### Remove old dataset
DATASET_FOLDER = f"{DATA_PATH}/training"

In [67]:
if path.exists(DATASET_FOLDER):
    for folder in listdir(DATASET_FOLDER):
        rmtree(path.join(DATASET_FOLDER, folder), ignore_errors=True)
# Make label folder
makedirs(path.join(DATASET_FOLDER, 'labels'), exist_ok=False)
# Make train, val, test folders
for folder in ['train', 'val', 'test']:
    makedirs(path.join(DATASET_FOLDER, 'images', folder), exist_ok=False)
    makedirs(path.join(DATASET_FOLDER, 'labels', folder), exist_ok=False)


In [70]:
# Get all component images from all folders
basenames = []
# For component folder in dataset
imgfiles = listdir(path.join(PARAM['path'], 'imgs'))
labfiles = listdir(path.join(PARAM['path'], 'labels'))
bases = [path.splitext(f)[0] for f in imgfiles]
basenames.extend(bases)
print(f"Found {len(imgfiles)} images and {len(labfiles)} labels.")

# Split the data into train, val, test
random.shuffle(basenames)
train = int(len(basenames) * PARAM['train'])
val = int(len(basenames) * PARAM['val'])
test = int(len(basenames) * PARAM['test'])
print(f"Split into {train} train, {val} val, {test} test. Total: {train+val+test} images.")
train_set = basenames[:train]
val_set = basenames[train:train+val]
test_set = basenames[train+val:]

# Copy the images and labels to the new dataset folder
for folder, dataset in zip(['train', 'val', 'test'], [train_set, val_set, test_set]):
    for base in dataset:
        filename = path.split(base)[1]
        copy(path.join(PARAM['path'], "imgs", f"{base}.png"), path.join(DATASET_FOLDER, 'images', folder, f"{filename}.png"))
        base = base.replace('imgs', 'labels')
        copy(path.join(PARAM['path'], "labels", f"{base}.txt"), path.join(DATASET_FOLDER, 'labels', folder, f"{filename}.txt"))

Found 257 images and 257 labels.
Split into 179 train, 51 val, 25 test. Total: 255 images.


In [68]:
# Tensorboard logging
%load_ext tensorboard
%tensorboard --logdir "logs" --port=6005

Reusing TensorBoard on port 6005 (pid 25652), started 5 days, 0:10:17 ago. (Use '!kill 25652' to kill it.)

In [ ]:
! taskkill /PID 40156 /F

In [76]:
# Train
model = YOLO(MODEL_PATH).to('cuda')
model.train(**TRAIN_PARAM)

New https://pypi.org/project/ultralytics/8.2.22 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.20  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
engine\trainer: task=detect, mode=train, model=./models/pretrained/yolov8n-coco.pt, data=./models/recipes/resistor_values.yaml, epochs=20, time=None, patience=10, batch=16, imgsz=128, save=True, save_period=5, cache=False, device=0, workers=8, project=logs, name=resistor_values_run-20240526-001155, exist_ok=False, pretrained=True, optimizer=auto, verbose=False, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False,

train: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\partial\resistor\training\labels\train.cache... 179 images, 0 backgrounds, 0 corrupt: 100%|██████████| 179/179 [00:00<?, ?it/s]
val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\partial\resistor\training\labels\val.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000588, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 128 train, 128 val
Using 8 dataloader workers
Logging results to logs\resistor_values_run-20240526-001155
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20     0.797G     0.8302      76.81      2.136         37        128: 100%|██████████| 12/12 [00:01<00:00,  6.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.94it/s]

                   all         51        323          0          0          0          0


...

                   all         51        323      0.828      0.327      0.409      0.197



20 epochs completed in 0.041 hours.
Optimizer stripped from logs\resistor_values_run-20240526-001155\weights\last.pt, 6.2MB
Optimizer stripped from logs\resistor_values_run-20240526-001155\weights\best.pt, 6.2MB

Validating logs\resistor_values_run-20240526-001155\weights\best.pt...
Ultralytics YOLOv8.2.20  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  6.94it/s]

                   all         51        323      0.828      0.327      0.409      0.197
Speed: 0.1ms preprocess, 0.7ms inference, 0.0ms loss, 1.6ms postprocess per image


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000000A30446D0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.0

In [77]:
# Test the model
metrics = model.val(data=TRAIN_PARAM['data'])

Ultralytics YOLOv8.2.20  Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3080 Laptop GPU, 16384MiB)
Model summary (fused): 168 layers, 3008183 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\partial\resistor\training\labels\val.cache... 51 images, 0 backgrounds, 0 corrupt: 100%|██████████| 51/51 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:05<00:00,  1.38s/it]

                   all         51        323      0.826      0.328      0.409      0.197
Speed: 0.2ms preprocess, 2.3ms inference, 0.0ms loss, 3.2ms postprocess per image


### Try Model

In [5]:
TEST_DATA = f"{DATA_PATH}/training/images/test"

def hex_to_bgr(hex):
    colour = tuple(int(hex[i:i+2], 16) for i in (0, 2, 4))
    return colour[::-1]

try:
    model
    print("Loaded trained model")
except:
    # Load the best model
    # List dir and find latest run
    runs = listdir(f"./logs")
    runs.sort()
    latest_run = runs[-1]
    modelPath = f"./logs/{latest_run}/weights/best.pt"
    print(f"Latest run: {latest_run}, loading model from {modelPath}")
    bestModel = YOLO(modelPath).to('cuda')
    model = bestModel
    print("Loaded best model")

classMap = {i: k for i, (k, v) in enumerate(DATA['resistors']['values'].items())} | {12 : "stem"}
colourMap = {i: hex_to_bgr(v[2][1::]) for i, (k, v) in enumerate(DATA['resistors']['values'].items())} | {12 : (179,179,0)}
print(colourMap)
print(classMap)
SCALE = 8
while cv2.waitKey(0) != ord('q'):
    cv2.destroyAllWindows()
    randomFile = random.choice(listdir(TEST_DATA))
    print(f"Random file: {randomFile}")
    results = model.predict(f"{TEST_DATA}/{randomFile}")
    # Need to upscale the image to show boxes
    img = cv2.imread(f"{TEST_DATA}/{randomFile}")
    img = cv2.resize(img, (img.shape[1]*SCALE, img.shape[0]*SCALE))
    for result in results:
        for i, _ in enumerate(result.boxes.cls):
            print(result)
            conf = result.boxes.conf[i]
            cls = result.boxes.cls[i].item()
            x1, y1, x2, y2 = result.boxes.xyxy[i]
            print(f"Conf: {conf}, Class: {cls}, Box: {x1, y1, x2, y2}")
            cv2.rectangle(img, (int(x1*SCALE), int(y1*SCALE)), (int(x2*SCALE), int(y2*SCALE)), colourMap[cls], 2)
            cv2.putText(img, f"{classMap[cls]}: {conf:.2f}", (int(x1*SCALE), int(y1*SCALE)), cv2.FONT_HERSHEY_SIMPLEX, 1, colourMap[cls], 2)
    cv2.imshow("Image", img)
cv2.destroyAllWindows()

Latest run: resistor_values_run-20240526-001155, loading model from ./logs/resistor_values_run-20240526-001155/weights/best.pt
Loaded best model
{0: (0, 0, 0), 1: (19, 69, 139), 2: (0, 0, 255), 3: (0, 165, 255), 4: (0, 255, 255), 5: (0, 255, 0), 6: (255, 0, 0), 7: (238, 130, 238), 8: (128, 128, 128), 9: (255, 255, 255), 10: (0, 215, 255), 11: (192, 192, 192), 12: (179, 179, 0)}
{0: 'black', 1: 'brown', 2: 'red', 3: 'orange', 4: 'yellow', 5: 'green', 6: 'blue', 7: 'violet', 8: 'grey', 9: 'white', 10: 'gold', 11: 'silver', 12: 'stem'}
Random file: obb_resistor_brown_white_black_black_brown_red_190E0-1_8.png

image 1/1 c:\Users\Shaheen\OneDrive - Imperial College London\Uni\CW Labs\Year 4\FYP\src\vision\datasets\partial\resistor\training\images\test\obb_resistor_brown_white_black_black_brown_red_190E0-1_8.png: 128x64 2 {0: 'black'}s, 2 {1: 'brown'}s, 1 {12: 'stem'}, 354.0ms
Speed: 7.5ms preprocess, 354.0ms inference, 2183.0ms postprocess per image at shape (1, 3, 128, 64)
ultralytics.engi

In [42]:
# need to rename class from 42 to 12
import re
for file in listdir("./datasets/partial/resistor/labels"):
    with open(f"./datasets/partial/resistor/labels/{file}", 'r') as f:
        lines = f.readlines()
    with open(f"./datasets/partial/resistor/labels/{file}", 'w') as f:
        for line in lines:
            f.write(re.sub(r"^42 ", "12 ", line))